## Imports

In [14]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from google.cloud import storage
import os
from io import StringIO

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [10]:
file_name = 'IWC_Work_Orders_Extract.csv'

df = pd.read_csv(file_name, low_memory=False)
print("File loaded from local storage.")

File loaded from local storage.


In [11]:
# change valid date columns to datetime
dates = ['EXECUTION_START_DATE', 'EXECUTION_FINISH_DATE', 'EQUIP_START_UP_DATE', 'EQUIP_VALID_FROM', 'EQUIP_VALID_TO']

for col in dates:
    df[col] = pd.to_datetime(df[col], errors='coerce')

In [12]:
df['START_YEAR_MONTH'] = pd.to_datetime(df['EXECUTION_START_DATE']).dt.strftime('%Y-%m')
df['MONTH'] = pd.to_datetime(df['EXECUTION_START_DATE']).dt.month
df['YEAR'] = pd.to_datetime(df['EXECUTION_START_DATE']).dt.year
df['WEEK'] = pd.to_datetime(df['EXECUTION_START_DATE']).dt.strftime('%W')

## Filtering

In [15]:
df.head()

,ORDER_ID,PLANT_ID,PRODUCTION_LOCATION,EXECUTION_START_DATE,EXECUTION_FINISH_DATE,ACTUAL_START_TIME,ACTUAL_FINISH_TIME,ACTUAL_WORK_IN_MINUTES,MAINTENANCE_PLAN,MAINTENANCE_ITEM,MAINTENANCE_ACTIVITY_TYPE,ORDER_DESCRIPTION,MAINTENANCE_TYPE_DESCRIPTION,FUNCTIONAL_LOC,FUNCTIONAL_AREA_NODE_1_MODIFIED,FUNCTIONAL_AREA_NODE_2_MODIFIED,FUNCTIONAL_AREA_NODE_3_MODIFIED,FUNCTIONAL_AREA_NODE_4_MODIFIED,FUNCTIONAL_AREA_NODE_5_MODIFIED,EQUIPMENT_ID,EQUIPMENT_DESC,EQUIP_CAT_DESC,EQUIP_START_UP_DATE,EQUIP_VALID_FROM,EQUIP_VALID_TO,START_YEAR_MONTH,MONTH,YEAR,WEEK
0,705642457,G812,ROMA,2024-05-04,2024-05-12,06:00:00.000,23:04:08.000,390.0,NaN,NaN,Unplanned,L2 GEARBOX LEAK (SEE ATTACHED),Corrective Maintenance Order,G812-PRD-L15-L03-L030-020,PRODUCTION,G812 PRD FILLER_ROTARY_CAN,G812 PRD FILLER_ROTARY_CAN LINE 3,CONVEYOR,FULL CAN CONVEYOR,300126812.0,FULL CAN CONVEYOR (FC41B),Machines,2020-04-22,2020-02-10,NaT,2024-05,5,2024,18
1,704191697,G812,ROMA,2022-09-13,2022-09-13,06:00:00.000,17:17:24.000,420.0,NaN,NaN,Unplanned,LINE 3 CONVEYOR AT CAN CODER,Corrective Maintenance Order,G812-PRD-L15-L03-L030-020,PRODUCTION,G812 PRD FILLER_ROTARY_CAN,G812 PRD FILLER_ROTARY_CAN LINE 3,CONVEYOR,FULL CAN CONVEYOR,300126812.0,FULL CAN CONVEYOR (FC41B),Machines,2020-04-22,2020-02-10,NaT,2022-09,9,2022,37
2,704466547,G812,ROMA,2022-12-21,2022-12-21,07:00:00.000,07:00:00.000,30.0,NaN,NaN,Unplanned,BROKEN CONVEYOR RAIL,Corrective Maintenance Order,G812-PRD-L15-L03-L030-020,PRODUCTION,G812 PRD FILLER_ROTARY_CAN,G812 PRD FILLER_ROTARY_CAN LINE 3,CONVEYOR,FULL CAN CONVEYOR,300126812.0,FULL CAN CONVEYOR (FC41B),Machines,2020-04-22,2020-02-10,NaT,2022-12,12,2022,51
3,703834477,G812,ROMA,2022-07-04,2022-07-04,06:00:00.000,06:00:00.000,60.0,NaN,NaN,Unplanned,LINE 3 CONVEYOR,Corrective Maintenance Order,G812-PRD-L15-L03-L030-020,PRODUCTION,G812 PRD FILLER_ROTARY_CAN,G812 PRD FILLER_ROTARY_CAN LINE 3,CONVEYOR,FULL CAN CONVEYOR,300126812.0,FULL CAN CONVEYOR (FC41B),Machines,2020-04-22,2020-02-10,NaT,2022-07,7,2022,27
4,704661125,G291,MONZA,2023-03-15,2023-03-15,07:00:00.000,07:00:00.000,120.0,NaN,NaN,Unplanned,REPLACED SONIC AIR BLOWER BELT,Corrective Maintenance Order,G291-PRD-P05,MONZA PRODUCTION,AIR SYSTEMS,NaN,NaN,NaN,300001088.0,TANK_STL_STOR_AIR,Machines,2017-02-15,2023-12-11,NaT,2023-03,3,2023,11


## Finding Aggregate Time Values


In [32]:
grouped_df = df.groupby(['PRODUCTION_LOCATION', 'MAINTENANCE_ACTIVITY_TYPE', 'FUNCTIONAL_AREA_NODE_2_MODIFIED', 'EQUIPMENT_ID']).agg(
    average_minutes = ('ACTUAL_WORK_IN_MINUTES', 'mean'),
    count = ('ACTUAL_WORK_IN_MINUTES', 'count')
).reset_index()

In [36]:
pivoted_df = grouped_df.pivot_table(
    index=['PRODUCTION_LOCATION', 'FUNCTIONAL_AREA_NODE_2_MODIFIED', 'EQUIPMENT_ID'], 
    columns='MAINTENANCE_ACTIVITY_TYPE', 
    values=['average_minutes', 'count']
).reset_index()

pivoted_df.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in pivoted_df.columns]

pivoted_df['time_saved'] = (pivoted_df['average_minutes_Unplanned'] - pivoted_df['average_minutes_Planned']) / pivoted_df['average_minutes_Unplanned'] * 100

In [44]:
aggregate_times = pivoted_df[(pivoted_df['count_Planned'] > 50) & (pivoted_df['count_Unplanned'] > 50) & (pivoted_df['time_saved'] > 0)].sort_values(by='time_saved', ascending=False)

In [45]:
aggregate_times.info()

<class 'pandas.core.frame.DataFrame'>
Index: 235 entries, 1622 to 49
Data columns (total 8 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   PRODUCTION_LOCATION_              235 non-null    object 
 1   FUNCTIONAL_AREA_NODE_2_MODIFIED_  235 non-null    object 
 2   EQUIPMENT_ID_                     235 non-null    float64
 3   average_minutes_Planned           235 non-null    float64
 4   average_minutes_Unplanned         235 non-null    float64
 5   count_Planned                     235 non-null    float64
 6   count_Unplanned                   235 non-null    float64
 7   time_saved                        235 non-null    float64
dtypes: float64(6), object(2)
memory usage: 16.5+ KB


## Finding Next Job

In [48]:
df_filtered = df[df['EQUIPMENT_ID'].isin(aggregate_times['EQUIPMENT_ID_'])]

df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 148499 entries, 662 to 1427263
Data columns (total 29 columns):
 #   Column                           Non-Null Count   Dtype         
---  ------                           --------------   -----         
 0   ORDER_ID                         148499 non-null  int64         
 1   PLANT_ID                         148499 non-null  object        
 2   PRODUCTION_LOCATION              148499 non-null  object        
 3   EXECUTION_START_DATE             148499 non-null  datetime64[ns]
 4   EXECUTION_FINISH_DATE            148499 non-null  datetime64[ns]
 5   ACTUAL_START_TIME                148499 non-null  object        
 6   ACTUAL_FINISH_TIME               148499 non-null  object        
 7   ACTUAL_WORK_IN_MINUTES           148499 non-null  float64       
 8   MAINTENANCE_PLAN                 73806 non-null   object        
 9   MAINTENANCE_ITEM                 73806 non-null   float64       
 10  MAINTENANCE_ACTIVITY_TYPE        148499 non-nu

In [56]:
selected_columns = ['PRODUCTION_LOCATION', 'FUNCTIONAL_AREA_NODE_2_MODIFIED', 'EQUIPMENT_ID', 'EXECUTION_START_DATE', 'EXECUTION_FINISH_DATE', 'ACTUAL_WORK_IN_MINUTES']
sm_df_filtered = df_filtered[selected_columns].sort_values(selected_columns)

# Aggregate the data
sm_df_filtered_agg = sm_df_filtered.groupby(['PRODUCTION_LOCATION', 'FUNCTIONAL_AREA_NODE_2_MODIFIED', 'EQUIPMENT_ID', 'EXECUTION_START_DATE', 'EXECUTION_FINISH_DATE']).agg(
    counts=('EQUIPMENT_ID', 'size'),
    total_mins=('ACTUAL_WORK_IN_MINUTES', 'sum')
).reset_index()

# Find the last execution date
sm_df_filtered_agg['PREV_EXECUTION_FINISH_DATE'] = sm_df_filtered_agg.groupby(['PRODUCTION_LOCATION', 'FUNCTIONAL_AREA_NODE_2_MODIFIED', 'EQUIPMENT_ID'])['EXECUTION_FINISH_DATE'].shift(1)

# Replace NaN values for the first occurrence
sm_df_filtered_agg['PREV_EXECUTION_FINISH_DATE'] = sm_df_filtered_agg['PREV_EXECUTION_FINISH_DATE'].fillna(sm_df_filtered_agg['EXECUTION_START_DATE'])

# Calculate lifespan (time difference in days between consecutive executions)
sm_df_filtered_agg['LIFE'] = (sm_df_filtered_agg['EXECUTION_START_DATE'] - sm_df_filtered_agg['PREV_EXECUTION_FINISH_DATE']).dt.days
sm_df_filtered_agg['LIFE'] = sm_df_filtered_agg['LIFE'].fillna(pd.NA)

# Compute weighted mean lifespan and mean time maintenance
workable_df = sm_df_filtered_agg
weighted_life = workable_df.groupby(['PRODUCTION_LOCATION', 'FUNCTIONAL_AREA_NODE_2_MODIFIED', 'EQUIPMENT_ID']).apply(
    lambda x: pd.Series({
        'WEIGHTED_MEAN_LIFESPAN': (x['LIFE'] * x['counts']).sum() / x['counts'].sum(),
        'MEAN_TIME_MAINTENENCE': x['total_mins'].mean()
    })
).reset_index()

# Select final columns from the original data
selected_columns = ['PRODUCTION_LOCATION', 'FUNCTIONAL_AREA_NODE_2_MODIFIED', 'EQUIPMENT_ID', 'EXECUTION_START_DATE', 'EXECUTION_FINISH_DATE', 'ACTUAL_WORK_IN_MINUTES', 'PREV_EXECUTION_FINISH_DATE', 'LIFE']
df_filtered_filtered = sm_df_filtered.filter(selected_columns)

# Merge the calculated weighted mean lifespan back to the original filtered dataset
df_lifespan = pd.merge(df_filtered_filtered, weighted_life, on=['PRODUCTION_LOCATION', 'FUNCTIONAL_AREA_NODE_2_MODIFIED', 'EQUIPMENT_ID'], how='left')

# Add the weighted mean lifespan as timedelta to the execution finish date
df_lifespan['EXPECTED_MAINTENANCE_DATE'] = df_lifespan['EXECUTION_FINISH_DATE'] + pd.to_timedelta(df_lifespan['WEIGHTED_MEAN_LIFESPAN'], unit='D')

# Ensure the EXPECTED_MAINTENANCE_DATE is in datetime format
df_lifespan['EXPECTED_MAINTENANCE_DATE'] = pd.to_datetime(df_lifespan['EXPECTED_MAINTENANCE_DATE'])

# Find the actual next maintenance date (next execution start date)
df_lifespan['ACTUAL_NEXT_MAINTENACE'] = pd.to_datetime(df_lifespan.groupby(['PRODUCTION_LOCATION', 'FUNCTIONAL_AREA_NODE_2_MODIFIED', 'EQUIPMENT_ID'])['EXECUTION_START_DATE'].shift(-1))

# Calculate the difference between the expected and actual maintenance dates
df_lifespan['EXPECTED_DIFFERENCE'] = (df_lifespan['EXPECTED_MAINTENANCE_DATE'] - df_lifespan['ACTUAL_NEXT_MAINTENACE']).dt.days

# Display the final dataset
df_lifespan

/var/folders/48/lqnzfz_50qx1c2hqrjm1fbs80000gn/T/ipykernel_32510/4232871044.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  weighted_life = workable_df.groupby(['PRODUCTION_LOCATION', 'FUNCTIONAL_AREA_NODE_2_MODIFIED', 'EQUIPMENT_ID']).apply(


,PRODUCTION_LOCATION,FUNCTIONAL_AREA_NODE_2_MODIFIED,EQUIPMENT_ID,EXECUTION_START_DATE,EXECUTION_FINISH_DATE,ACTUAL_WORK_IN_MINUTES,WEIGHTED_MEAN_LIFESPAN,MEAN_TIME_MAINTENENCE,EXPECTED_MAINTENANCE_DATE,ACTUAL_NEXT_MAINTENACE,EXPECTED_DIFFERENCE
0,COTA,BOTTLE LINE,300025772.0,2017-11-01,2017-11-01,45.0,8.966777,47.546182,2017-11-09 23:12:09.568106314,2017-11-26,-17.0
1,COTA,BOTTLE LINE,300025772.0,2017-11-26,2017-11-26,30.0,8.966777,47.546182,2017-12-04 23:12:09.568106314,2017-11-29,5.0
2,COTA,BOTTLE LINE,300025772.0,2017-11-29,2017-11-29,30.0,8.966777,47.546182,2017-12-07 23:12:09.568106314,2017-11-30,7.0
3,COTA,BOTTLE LINE,300025772.0,2017-11-30,2017-11-30,24.0,8.966777,47.546182,2017-12-08 23:12:09.568106314,2017-12-07,1.0
4,COTA,BOTTLE LINE,300025772.0,2017-12-07,2017-12-07,15.0,8.966777,47.546182,2017-12-15 23:12:09.568106314,2017-12-07,8.0
...,...,...,...,...,...,...,...,...,...,...,...
148494,SUZUKA,L4 - FILLER_ROTARY_CAN,300009666.0,2024-06-18,2024-06-18,30.0,11.834783,109.246154,2024-06-29 20:02:05.217391308,2024-07-29,-30.0
148495,SUZUKA,L4 - FILLER_ROTARY_CAN,300009666.0,2024-07-29,2024-07-29,60.0,11.834783,109.246154,2024-08-09 20:02:05.217391308,2024-08-12,-3.0
148496,SUZUKA,L4 - FILLER_ROTARY_CAN,300009666.0,2024-08-12,2024-08-12,180.0,11.834783,109.246154,2024-08-23 20:02:05.217391308,2024-08-13,10.0
148497,SUZUKA,L4 - FILLER_ROTARY_CAN,300009666.0,2024-08-13,2024-08-13,30.0,11.834783,109.246154,2024-08-24 20:02:05.217391308,2024-08-24,0.0
